In [1]:
from lxml import etree
import pandas as pd
from tqdm import tqdm
import os



In [5]:
#setup
path = 'output_xml 2023-03-15 BR'
xml_file = 'DFT_20230316-121715-All Contracts.xml'
excel_file = '2023-03-15 BR MN Template for External Contracts - Service and Rental v1.9 - Brazil 2023-03-15.xlsx'
#'2023-03-15 BR MN Template for External Contracts - Service and Rental v1.9 - Brazil 2023-03-15.xlsx'
sheet_name = 'Service and Rental - TEMPLATE'
#'Tiered Line Item - TEMPLATE'
#

#load_file = '2023-03-06 BR Load Results.xlsx'

In [6]:
# Opening excel and xml file
tree = etree.parse(f'{path}/{xml_file}')
root=tree.getroot()
excel = pd.read_excel(f'{path}/{excel_file}', header = 3, sheet_name=f'{sheet_name}')
df_log = pd.read_csv(f'{path}/logs.csv', header=None)

In [7]:
excel["Product ID"]

0       02G3302
1       02G3302
2       03R6599
3       03R6599
4       01DL309
         ...   
1116    01DC279
1117    01DL301
1118    01DL601
1119    02G3303
1120    01DL602
Name: Product ID, Length: 1121, dtype: object

In [8]:
# saving removed products as list
df_log.columns=['t', 'd', 'a', 'c']
df_log[['A', 'B']] = df_log['a'].str.split(': ', 1, expand=True)
removed = df_log[['B']]
removed[['A', 'B']] = df_log['B'].str.split('_', 1, expand=True)

######## service and rental ########
removed['A'] = removed['A'].str.replace("-", "")

removed = removed['A']
removed = list(removed.drop_duplicates(keep='last'))

/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1313/2894553172.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  removed[['A', 'B']] = df_log['B'].str.split('_', 1, expand=True)
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1313/2894553172.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  removed['A'] = removed['A'].str.replace("-", "")


In [152]:
removed

['09P27-97',
 '09P36-97',
 '09P38-97',
 '09P48-97',
 '09P92-97',
 '08P22-97',
 '01R06-97',
 '09P85-97',
 '09P34-97',
 '09P35-97',
 '09P39-97',
 '09P37-97',
 '04T75-97',
 '09P42-97',
 '09P21-97',
 '09P22-97',
 '09P20-97',
 '01R18-97',
 '07P58-97',
 '09P56-97',
 '09P57-97',
 '09P95-97',
 '09D28-97',
 '01R16-97',
 '09P61-97',
 '01R13-97',
 '09P63-97',
 '04Y85-97',
 '07P52-97',
 '08P52-97',
 '09P62-97',
 'PPC',
 '09P19-97',
 '02G69-01',
 '02G49-07',
 '02P25-97',
 '02G55-21',
 '02G49-01',
 '02G49-04',
 '02G49-08',
 '02G61-24',
 '02G49-06',
 '02G49-11',
 '02G49-09',
 '06P09-97',
 '06P08-97',
 '06P07-97',
 '06P04-97',
 '06P06-97',
 '05N23-97',
 '07P79-97',
 '07P80-97',
 '09H62-97',
 '04J27-30',
 '04T86-30']

In [9]:
# removing products from log from excel
for index in tqdm(excel.index):
    if excel['Product ID'][index] in removed:
        excel.drop([index], axis=0, inplace=True)

100%|██████████| 1121/1121 [00:00<00:00, 176038.59it/s]


In [10]:
# count of products
excel_gr = excel.groupby(['Alternate Contract Id','Customer ID', 
'Price Program Name']).size().reset_index(name='counts')


In [43]:
# Getting numbers and list with products
number_contracts = 0
number_lines = 0

list_all_extended = []
Contract = tree.xpath('.//ContractOffer')
for contract in Contract:
    number_contracts += 1
    Program = contract.xpath('.//Program')
    number_programs = 0
    for program in Program:
        number_programs += 1
        Cfd = program.xpath('.//CFDefinitionSpreadSheets')
        number_products = 0
        # #tierdli
        # if program.attrib['SourceStrategyID'] == 'AbbottTieredLineItemPricingStrategy':
        #     Product = program.xpath('.//ProductLI')
        #     for product in Product:
        #         number_products += 1
        #         number_lines += 1
        #         Price = product.xpath('.//TieredLIPrices')
        #         for price in Price:
        #             try:
        #                 list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
        #                 contract.attrib['OrgUnitName'],contract.attrib['DocName'], contract.attrib['Currency'],
        #                 program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['StartDate'],
        #                 program.attrib['ContractStartDate'],program.attrib['EndDate'],
        #                 product.attrib['ProductNum'], 
        #                 product.attrib['StartEffectiveDate'], product.attrib['EndEffectiveDate'],
        #                 price.attrib['PriceHigh'], price.attrib['TierVolume']])

        #             except:
        #                 list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
        #                 contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
        #                 program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['StartDate'],
        #                 program.attrib['ContractStartDate'],program.attrib['EndDate'],
        #                 product.attrib['ProductNum'],
        #                 product.attrib['StartEffectiveDate'], product.attrib['EndEffectiveDate'],
        #                 price.attrib['AltUomPrice'], price.attrib['TierVolume']])
     
        # elif program.attrib['SourceStrategyID'] == 'AbbottDOLPricingStrategy':
        #     Discount = program.xpath('.//DiscountOffListLI')
        #     for discount in Discount:
        #         number_products += 1
        #         number_lines += 1
        #         Price = discount.xpath('.//LIPrices')
        #         for price in Price:
        #             try:
        #                 list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
        #                 contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
        #                 program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['StartDate'],
        #                 program.attrib['ContractStartDate'],program.attrib['EndDate'],
        #                 discount.attrib['ProductNum'],
        #                 discount.attrib['StartEffectiveDate'], discount.attrib['EndEffectiveDate'],
        #                 price.attrib['PriceHigh'], ''])

        #             except:
        #                 list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
        #                 contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
        #                 program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['StartDate'],
        #                 program.attrib['ContractStartDate'],program.attrib['EndDate'],
        #                 discount.attrib['ProductNum'],
        #                 discount.attrib['StartEffectiveDate'], discount.attrib['EndEffectiveDate'],
        #                 price.attrib['AltUomPrice'], ''])

        #service i rental
        if program.attrib['SourceStrategyID'] in ['AbbottRentalPricingStrategy','AbbottServicePricingStrategy', 'AbbottSubscriptionPricingStrategy']:
            Rental = program.xpath('.//RentalLI')
            for rental in Rental:
                number_products += 1
                number_lines += 1
                Price = rental.xpath('.//LIPrices')
                Instrument = rental.xpath('.//InstrumentProduct')
               
                for price in Price:
                    for instrument in Instrument:
                        list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
                        contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
                        program.attrib['Name'], program.attrib['BusinessSegment'],program.attrib['StartDate'],
                        program.attrib['ContractStartDate'],program.attrib['EndDate'],
                        instrument.attrib['ProductNum'],
                        rental.attrib['ProductNum'], 
                        rental.attrib['StartEffectiveDate'], rental.attrib['EndEffectiveDate'],
                        price.attrib['PriceHigh']])

                    # list_all_extended.append([contract.attrib['Customer'], contract.attrib['AlternateId'],contract.attrib['StartDate'],contract.attrib['EndDate'],
                    # contract.attrib['OrgUnitName'],contract.attrib['DocName'],contract.attrib['Currency'],
                    # program.attrib['Name'], program.attrib['BusinessSegment'], program.attrib['ContractStartDate'],program.attrib['EndDate'],
                    # '',
                    # rental.attrib['ProductNum'], 
                    # rental.attrib['StartEffectiveDate'], rental.attrib['EndEffectiveDate'],
                    # price.attrib['PriceHigh']])
                
        

                    
  

In [44]:
# dataframes
df_all_extended = pd.DataFrame(list_all_extended)
df_all_extended.columns =['Customer ID','Alternate Contract Id', 'Contract Start Date','Contract End Date',
'Organization', 'Name', 'Currency',
'Price Program Name', 'Business Segment','C Start Date', 
'Program Start Date','Program End Date',
'Instrument #',
'Product ID', 'Product Start Date', 'Product End Date', 
# for rental
'Price']
# for tieredli
#,'Units']
excel_extended = excel[['Customer ID','Alternate Contract Id', 'Start Date', 'End Date',
'Organization', 'Name','Currency',
'Price Program Name', 'Business Segment', 'Program Start Date', 'Program End Date',
'Instrument #',
 'Product ID', 'Product Start Date', 'Product End Date', 
 # for rental
 'Price']]
 # for tieredli
 #'Tier 1 Price' , 'Tier 1 Units']]


In [18]:
# number of all lines
print(excel.shape[0])
print(df_all_extended.shape[0])
print(len(removed))

1121
1121
55


In [45]:
# removing price programs and contract ids with -2
for line in tqdm(df_all_extended.index):
    if df_all_extended['Price Program Name'][line][-2:] == '-2':
        df_all_extended['Price Program Name'][line] = df_all_extended['Price Program Name'][line].replace("-2", "")
    if df_all_extended['Alternate Contract Id'][line][-2:] == '-2':
       df_all_extended['Alternate Contract Id'][line] = df_all_extended['Alternate Contract Id'][line].replace("-2", "")



100%|██████████| 1121/1121 [00:00<00:00, 131079.31it/s]


In [46]:
# extended
df_all_extended['Price'] = df_all_extended.Price.astype(str).str.extract(r"(\d+\.\d+)")
df_all_extended['Price'] = df_all_extended['Price'].astype(float)
# for service and rental
#excel_extended['Tier 1 Price'] = excel_extended['Tier 1 Price'].astype(float)

# for tieredli
#excel_extended['Tier 1 Price'] = excel_extended['Tier 1 Price'].astype(float)
#df_all_extended['Units'] = pd.to_numeric(df_all_extended['Units'], downcast="float")
#excel_extended['Tier 1 Units'] = pd.to_numeric(excel_extended['Tier 1 Units'], downcast="float")

In [176]:
df_merged_extended['Units'] = pd.to_numeric(df_merged_extended['Units'])
df_merged_extended['Tier 1 Units'] = pd.to_numeric(df_merged_extended['Tier 1 Units'])

In [47]:
excel_extended['Instrument #'] = excel_extended['Instrument #'].str.replace("-", "")


/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1313/3472975014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Instrument #'] = excel_extended['Instrument #'].str.replace("-", "")


In [39]:
# Merging dataframes
excel_extended['Customer ID'] = excel_extended['Customer ID'].astype(int)
df_all_extended['Customer ID'] = df_all_extended['Customer ID'].astype(int)

#service and rental
df_all_extended['Product ID'] = df_all_extended['Product ID'].str.replace("-", "")

df_merged_extended = pd.merge(df_all_extended, excel_extended,  how='outer', left_on=['Customer ID','Alternate Contract Id', 'Price Program Name', 'Product ID',
'Instrument #'
], 
right_on = ['Customer ID', 'Alternate Contract Id', 'Price Program Name','Product ID',
'Instrument #'
])


/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1313/1280410995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excel_extended['Customer ID'] = excel_extended['Customer ID'].astype(int)


In [168]:
# count of products
df_gr = df_all_extended.groupby(['Alternate Contract Id','Customer ID', 
'Price Program Name']).size().reset_index(name='counts')

In [169]:
# Merging dataframes- counts
excel_gr['Customer ID'] = excel_gr['Customer ID'].astype(int)
df_gr['Customer ID'] = df_gr['Customer ID'].astype(int)

df_merged = pd.merge(df_gr, excel_gr,  how='outer', left_on=['Customer ID','Alternate Contract Id', 'Price Program Name'], right_on = ['Customer ID','Alternate Contract Id', 'Price Program Name'])
df_merged.rename(columns = {'counts_x':'count from xml', 'counts_y':'count from excel'}, inplace = True)

In [170]:
# checking if the same
df_merged['Are the same?'] = ''
for index in tqdm(df_merged.index):
    if df_merged['count from xml'][index] == df_merged['count from excel'][index]:
        df_merged['Are the same?'][index] = 'yes'
    else:
        df_merged['Are the same?'][index] = 'no'

  0%|          | 0/1691 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1425/189006053.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Are the same?'][index] = 'yes'
100%|██████████| 1691/1691 [00:00<00:00, 11850.45it/s]


In [171]:
df_merged.to_excel(f'{path}/result counts.xlsx')

In [48]:
# data types 
df_merged_extended['Product Start Date_x']=pd.to_datetime(df_merged_extended['Product Start Date_x'])
df_merged_extended['Product Start Date_y']=pd.to_datetime(df_merged_extended['Product Start Date_y'])
df_merged_extended['Product End Date_x']=pd.to_datetime(df_merged_extended['Product End Date_x'])
df_merged_extended['Product End Date_y']=pd.to_datetime(df_merged_extended['Product End Date_y'])
df_merged_extended['Program Start Date_x']=pd.to_datetime(df_merged_extended['Program Start Date_x'])
df_merged_extended['Program Start Date_y']=pd.to_datetime(df_merged_extended['Program Start Date_y'])
df_merged_extended['Program End Date_x']=pd.to_datetime(df_merged_extended['Program End Date_x'])
df_merged_extended['Program End Date_y']=pd.to_datetime(df_merged_extended['Program End Date_y'])
df_merged_extended['Contract Start Date']=pd.to_datetime(df_merged_extended['Contract Start Date'])
df_merged_extended['C Start Date']=pd.to_datetime(df_merged_extended['C Start Date'])
df_merged_extended['Start Date']=pd.to_datetime(df_merged_extended['Start Date'])
df_merged_extended['Contract End Date']=pd.to_datetime(df_merged_extended['Contract End Date'])
df_merged_extended['End Date']=pd.to_datetime(df_merged_extended['End Date'])
df_merged_extended['Organization_y']= pd.to_numeric(df_merged_extended['Organization_y'])
df_merged_extended['Organization_x']= pd.to_numeric(df_merged_extended['Organization_x'])

In [49]:
df_merged_extended['Are prices the same?'] = ''
df_merged_extended['Are start dates the same?'] = ''
df_merged_extended['Are end dates the same?'] = ''
df_merged_extended['Is org the same?'] = ''
df_merged_extended['Is name the same?'] = ''
df_merged_extended['Is contract program start date the same?'] = ''
df_merged_extended['Is program start date the same?'] = ''
df_merged_extended['Is program end date the same?'] = ''
df_merged_extended['Is product start date the same?'] = ''
df_merged_extended['Is product end date the same?'] = ''
df_merged_extended['Are units the same?'] = ''
df_merged_extended['Is instrument the same?']=''
for index in tqdm(df_merged_extended.index):
    #tieredli
   # if df_merged_extended['Price'][index] == df_merged_extended['Tier 1 Price'][index]:
    #rental
    if df_merged_extended['Price_x'][index] == df_merged_extended['Price_y'][index]:
        df_merged_extended['Are prices the same?'][index] = 'yes'
    else:
        df_merged_extended['Are prices the same?'][index] = 'no'

    if df_merged_extended['Contract Start Date'][index] == df_merged_extended['Start Date'][index]:
        df_merged_extended['Are start dates the same?'] = 'yes'
    else:
        df_merged_extended['Are start dates the same?'] = 'no'

    if df_merged_extended['Contract End Date'][index] == df_merged_extended['End Date'][index]:
        df_merged_extended['Are end dates the same?'][index] = 'yes'
    else:
        df_merged_extended['Are end dates the same?'][index] = 'no'

    if df_merged_extended['Organization_x'][index] == df_merged_extended['Organization_y'][index]:
        df_merged_extended['Is org the same?'][index] = 'yes'
    else:
        df_merged_extended['Is org the same?'][index] = 'no'

    if df_merged_extended['Name_x'][index] == df_merged_extended['Name_y'][index]:
        df_merged_extended['Is name the same?'][index] = 'yes'
    else:
        df_merged_extended['Is name the same?'][index] = 'no'

    if df_merged_extended['Product Start Date_x'][index] == df_merged_extended['Product Start Date_y'][index]:
        df_merged_extended['Is product start date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is product start date the same?'][index] = 'no'

    if df_merged_extended['Product End Date_x'][index] == df_merged_extended['Product End Date_y'][index]:
        df_merged_extended['Is product end date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is product end date the same?'][index] = 'no'

    if df_merged_extended['Program Start Date_x'][index] == df_merged_extended['Program Start Date_y'][index]:
            df_merged_extended['Is program start date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is program start date the same?'][index] = 'no'

    
    if df_merged_extended['Program End Date_x'][index] == df_merged_extended['Program End Date_y'][index]:
            df_merged_extended['Is program end date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is program end date the same?'][index] = 'no'

    if df_merged_extended['C Start Date'][index] == df_merged_extended['Program Start Date_y'][index]:
            df_merged_extended['Is contract program start date the same?'][index] = 'yes'
    else:
        df_merged_extended['Is contract program start date the same?'][index] = 'no'

    # if df_merged_extended['Instrument #_x'][index] == df_merged_extended['Instrument #_y'][index]:
    #     df_merged_extended['Is instrument the same?'][index] = 'yes'
    # else:
    #         df_merged_extended['Is instrument the same?'][index] = 'no'




    # if df_merged_extended['Units'][index] == df_merged_extended['Tier 1 Units'][index]:
    #     df_merged_extended['Are units the same?'][index] = 'yes'
    # else:
    #     df_merged_extended['Are units the same?'][index] = 'no'






  0%|          | 0/3466 [00:00<?, ?it/s]/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1313/2886323307.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are prices the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1313/2886323307.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_extended['Are end dates the same?'][index] = 'yes'
/var/folders/b3/52rfzyl51kv8hp5lk3vw_b6m0000gn/T/ipykernel_1313/2886323307.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [50]:

df_merged_extended.to_excel(f'{path}/result extended tieredli.xlsx')